In [1]:
import os
import sys
import time
import openpyxl

import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from datetime import datetime

sys.path.append('../')

from tsgen import TsGenerator
from nc_explorer import NcExplorer

tsgen = TsGenerator()

sen3r-NcExplorer: from mpl_toolkits.basemap import Basemap FAILED!
You can still proceed without plotting any maps.



## Extracting RAW .SEN3 NetCDF4 without SNAP

In [2]:
# stations = {
#     '/d_drive_data/S3/stations/12350000_fonte_boa/': ['/d_drive_data/S3/stations/12350000_fonte_boa_img/','/d_drive_data/A1_JM/areas/new_areas/12350000_fonte_boa.geojson'],
#     '/d_drive_data/S3/stations/12900001_tefe/': ['/d_drive_data/S3/stations/12900001_tefe_img/','/d_drive_data/A1_JM/areas/new_areas/12900001_tefe.geojson'],
#     '/d_drive_data/S3/stations/13100090_coari/': ['/d_drive_data/S3/stations/13100090_coari_img/','/d_drive_data/A1_JM/areas/new_areas/13100090_coari.geojson'],
#     '/d_drive_data/S3/stations/15860000_borba_madeira/': ['/d_drive_data/S3/stations/15860000_borba_madeira_img/','/d_drive_data/A1_JM/areas/new_areas/15860000_borba_madeira.geojson'],
#     '/d_drive_data/S3/stations/17050001_obidos/': ['/d_drive_data/S3/stations/17050001_obidos_img/','/d_drive_data/A1_JM/areas/new_areas/17050001_obidos.geojson']
#     '/d_drive_data/S3/stations/14910000_negro/': ['/d_drive_data/S3/stations/','/d_drive_data/A1_JM/areas/new_areas/14910000_negro.geojson'],
#     '/d_drive_data/S3/stations/14900050_negro_2/': ['/d_drive_data/S3/stations/','/d_drive_data/A1_JM/areas/new_areas/14900050_negro_2.geojson']
# }
stations = {
    '/d_drive_data/S3/stations/AN1/': ['/d_drive_data/S3/stations/','/d_drive_data/A1_JM/artigo_SEN3R/ROIs/AN1.geojson'],
    '/d_drive_data/S3/stations/BCO/': ['/d_drive_data/S3/stations/','/d_drive_data/A1_JM/artigo_SEN3R/ROIs/BCO.geojson']
#     '/d_drive_data/S3/stations/14100000_manacapuru/': ['/d_drive_data/S3/stations/14100000_manacapuru/','/d_drive_data/A1_JM/areas/paper_areas/14100000_manacapuru.geojson']
#     '/d_drive_data/S3/stations/14900050_negro_19/': ['/d_drive_data/S3/stations/','/d_drive_data/A1_JM/areas/paper_areas/14900050_negro_19.geojson'],
#     '/d_drive_data/S3/stations/14900050_negro_29/': ['/d_drive_data/S3/stations/','/d_drive_data/A1_JM/areas/paper_areas/14900050_negro_29.geojson'],
#     '/d_drive_data/S3/stations/14900050_negro_37/': ['/d_drive_data/S3/stations/','/d_drive_data/A1_JM/areas/paper_areas/14900050_negro_37.geojson']
}

In [ ]:
for s in stations:
    
    # LINUX
    in_dir = '/d_drive_data/S3/L2_WFR'
    out_dir = s
    img_dir = stations[s][0]
    poly = stations[s][1]
    
    todo = tsgen.build_list_from_subset(in_dir)
    todo_fullpath = [os.path.join(in_dir,csv) for csv in todo]
    total = len(todo_fullpath)
    
    info_string = f'Processing {total} files for station: {s} using polygon: {stations[s][1]}'
    print(info_string)
    os.system(f'telegram-send \"{info_string}\"')
    
    t1 = time.perf_counter()

    for n, img in enumerate(todo_fullpath):

        f_b_name = os.path.basename(img).split('.')[0]
        print(f'>>> Processing: {n+1} of {total} ... {f_b_name}')
        
        if os.path.isfile(os.path.join(out_dir, f_b_name+'.csv')):
            print('Skipped.')
            continue
        
        ncxp = NcExplorer(input_nc_folder=img, product='wfr')

        df = ncxp.get_data_in_poly(poly_path=poly, go_parallel=False)

        # if df is not None:
        print(f'Saving DF: {f_b_name}')
        df.to_csv(os.path.join(out_dir, f_b_name+'.csv'), index=False)

    t2 = time.perf_counter()
    outputstr = f'>>> Finished in {round(t2 - t1, 2)} second(s). <<<'
    print(outputstr)
    os.system(f'telegram-send \"{outputstr}\"')

## Generating post-processed CSVs from the raw extracted CSVs

In [3]:
version = 15

* Folder of files containing raw pixels extracted with GPT/SEN3R
* Folder to save the processed files
* Folder in which the plots should be saved

In [4]:
# # retrieve all files in folder
# shps_path = '/d_drive_data/A1_JM/areas/paper_areas/shp'
# files = os.listdir(shps_path)
# # extract only the Shapefiles from the list
# shps = [os.path.join(shps_path,f) for f in files if f.endswith('.shp')]
# shps

# shps = ['/d_drive_data/A1_JM/areas/paper_areas/shp/14910000_negro.shp']
# shps = ['/d_drive_data/A1_JM/areas/paper_areas/shp/14100000_manacapuru.shp']
# shps = ['/d_drive_data/A1_JM/areas/paper_areas/shp/14900050_negro_19.shp',
#         '/d_drive_data/A1_JM/areas/paper_areas/shp/14900050_negro_29.shp',
#         '/d_drive_data/A1_JM/areas/paper_areas/shp/14900050_negro_37.shp']
shps = ['/d_drive_data/A1_JM/artigo_SEN3R/ROIs/SHP/AN1.shp',
        '/d_drive_data/A1_JM/artigo_SEN3R/ROIs/SHP/BCO.shp']
shps

['/d_drive_data/A1_JM/artigo_SEN3R/ROIs/SHP/AN1.shp',
 '/d_drive_data/A1_JM/artigo_SEN3R/ROIs/SHP/BCO.shp']

In [ ]:
raw_dir = '/d_drive_data/S3/stations/'
dest = '/d_drive_data/processing/linux/'

for station in shps:
    
    procstring = f'processing: {station}'
    print(procstring)
    os.system(f'telegram-send \"{procstring}\"')
    
#     # GET SHP CENTROIDS
#     polygons = gpd.GeoDataFrame.from_file(station)
#     polygons.geometry = polygons.representative_point()
#     centroids = []
#     for g in polygons['geometry']:
#         q_lon, q_lat = g.coords[:][0]
#         centroids.append((q_lon, q_lat))
    
 
    # GET SERIES SAVE PATH
    station_name =    os.path.basename(station).split('.')[0]
    in_dir =          os.path.join(raw_dir,station_name)
    out_dir =         os.path.join(dest,station_name+f'_v{version}')
    img_dir =         os.path.join(dest,station_name+f'_v{version}_img')
    img_save_pth =    os.path.join(dest,station_name+f'_v{version}_img_dbscan')
    series_save_pth = os.path.join(dest,station_name+f'_v{version}_img_dbscan_series')


    # CREATE THE DIRECTORIES IF THEY DOESN'T EXIST YET
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    Path(img_dir).mkdir(parents=True, exist_ok=True)
    Path(img_save_pth).mkdir(parents=True, exist_ok=True)
    Path(series_save_pth).mkdir(parents=True, exist_ok=True)


    # GET LIST OF CSV FILES
    todo = tsgen.build_list_from_subset(in_dir)
    todo_fullpath = [os.path.join(in_dir,csv) for csv in todo]
    
    # Start timer
    t1 = time.perf_counter()
    
    # Desclaring lists for further validation
    checklist = {}
    skiplst = []
    donelst = []
    
    # irmax = 0.001 # Negro
    irmax = 0.2 # Generic
    # irmax = 0.08 # Fonte Boa
    # irmin = 0.001 # Manacapuru


    total = len(todo_fullpath)
    for n,img in enumerate(todo_fullpath):
        print(f'>>> Processing: {n+1} of {total} ... {img}')

        # reprocessing the raw CSVs and removing 
        # reflectances above the threshold in IR.

        try:
            dfpth, df = tsgen.update_csvs(csv_path=img,
                                          glint=20.0,
                                          #ir_min_threshold=irmin,
                                          ir_max_threshold=irmax,
                                          savepath=out_dir)
        except Exception as e:
            print("type error: " + str(e))
            skiplst.append(img)
            continue

        # The KDE needs at least two different reflectance values to work.
        if dfpth == 'KDE_fail':
            skiplst.append(img)
            continue

    #     try:
    #         dfpth, df = tsgen.update_csvs(csv_path=img,savepath=out_dir,threshold=0.2)
    #     except Exception as e:
    #         print("type error: " + str(e))
    #         print(f'Skipping CSV for lack of data: {figdate}')
    #         skiplst.append(img)
    #         continue

    #     dfpth, df = tsgen.update_csvs(csv_path=img,
    #                                   savepath=out_dir,
    #                                   threshold=0.2)

        if len(df) < 1:
            print(f'Skipping empty CSV: {dfpth}')
            skiplst.append(img)
            continue
        else:
            donelst.append(img)


        # fig params -------------------------------------------------
        figdate = os.path.basename(img).split('____')[1].split('_')[0]
        figtitl = os.path.basename(out_dir)+'_'+figdate
        savpt_sctr = os.path.join(img_dir,figdate+'_0.png')
        savpt_rrs = os.path.join(img_dir,figdate+'_1.png')

        # generate plot v1 --------------------------
    #     tsgen.plot_single_sktr(
    #         xdata=df['Oa08_reflectance:float'],
    #         ydata=df['Oa21_reflectance:float'],
    #         xlabel='Oa08: 665 nm',
    #         ylabel='Oa17: 865 nm',
    #         color=df['T865:float'],
    #         clabel='T865',
    #         title=figtitl,
    #         savepathname=savpt)

        # generate plot v3 --------------------------
        tsgen.plot_sidebyside_sktr(x1_data=df['Oa08_reflectance:float'],
                                   y1_data=df['Oa17_reflectance:float'],
                                   x2_data=df['Oa08_reflectance:float'],
                                   y2_data=df['Oa17_reflectance:float'],
                                   x_lbl='RED: Oa08 (665nm)',
                                   y_lbl='NIR: Oa17 (865nm)',
                                   c1_data=df['A865:float'],
                                   c1_lbl='Aer. Angstrom Expoent (A865)',
                                   c2_data=df['T865:float'],
                                   c2_lbl='Aer. Optical Thickness (T865)',
                                   title=f'{os.path.basename(out_dir)} WFR {figdate} RED:Oa08(665nm) x NIR:Oa17(865nm)',
                                   savepathname=savpt_sctr)

        tsgen.s3l2_custom_reflectance_plot(df=df,
                                           figure_title=figdate,
                                           c_lbl='Aer. Optical Thickness (T865)',
                                           save_title=savpt_rrs)

    t2 = time.perf_counter()
    outputstr = f'>>> Finished in {round(t2 - t1, 2)} second(s). <<<'
    print(outputstr)
    os.system(f'telegram-send \"{outputstr}\"')
    
    # Generating excel file from the post-processed data
    wdir = out_dir
    todo = tsgen.build_list_from_subset(wdir)
    todo_fullpath = [os.path.join(wdir,txt) for txt in todo]
    
    # Converting and saving the list of mean values into a XLS excel file.
    data = tsgen.generate_time_series_data(wdir,todo)

    series_df = pd.DataFrame(data=data)
    # Delete these row indexes from dataFrame
    # indexNames = series_df[series_df['B17-865'] > irmax].index
    # indexNames = series_df[series_df['B17-865'] < irmin].index
    # series_df.drop(indexNames, inplace=True)
    
    # create empty excel
    path = os.path.join(dest,station_name+f'_v{version}.xlsx')
    wb = openpyxl.Workbook()
    wb.save(path)

    # open the empty file and fill it up
    sheet = f'v{version}'
    book = openpyxl.load_workbook(path)
    writer = pd.ExcelWriter(path, engine = 'openpyxl')
    writer.book = book

    # LINUX
    series_df.to_excel(writer, sheet_name=sheet, index=False)

    writer.save()
    writer.close()

In [ ]:
# Generating excel file from the post-processed data
wdir = out_dir
todo = tsgen.build_list_from_subset(wdir)
todo_fullpath = [os.path.join(wdir,txt) for txt in todo]

# Converting and saving the list of mean values into a XLS excel file.
data = tsgen.generate_time_series_data(wdir,todo)

series_df = pd.DataFrame(data=data)
# Delete these row indexes from dataFrame
# indexNames = series_df[series_df['B17-865'] > irmax].index
# indexNames = series_df[series_df['B17-865'] < irmin].index
# series_df.drop(indexNames, inplace=True)

# create empty excel
path = os.path.join(dest,station_name+f'_v{version}.xlsx')
wb = openpyxl.Workbook()
wb.save(path)

# open the empty file and fill it up
sheet = f'v{version}'
book = openpyxl.load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book

# LINUX
series_df.to_excel(writer, sheet_name=sheet, index=False)

writer.save()
writer.close()

In [ ]:
series_df.columns

In [ ]:
# open the empty file and fill it up
sheet = f'v{version}'
book = openpyxl.load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book

# LINUX
series_df.to_excel(writer, sheet_name=sheet, index=False)

writer.save()
writer.close()

## Generating new CSVs and plots based in the RAW data.

In [ ]:
# dfpth, df = tsgen.update_csvs(csv_path=todo_fullpath[0],savepath=out_dir,threshold=0.2)

In [ ]:
t1 = time.perf_counter()
skiplst = []
donelst = []

total = len(todo_fullpath)
for n,img in enumerate(todo_fullpath):
    print(f'>>> Processing: {n+1} of {total} ... {img}')
    
    # reprocessing the raw CSVs and removing 
    # reflectances above the threshold in IR.
    
    
    try:
        dfpth, df = tsgen.update_csvs(csv_path=img,savepath=out_dir,threshold=0.2,glint=20.0)
    except Exception as e:
        print("type error: " + str(e))
        skiplst.append(img)
        continue

    # The KDE needs at least two different reflectance values to work.
    if dfpth == 'KDE_fail':
        skiplst.append(img)
        continue
    
#     try:
#         dfpth, df = tsgen.update_csvs(csv_path=img,savepath=out_dir,threshold=0.2)
#     except Exception as e:
#         print("type error: " + str(e))
#         print(f'Skipping CSV for lack of data: {figdate}')
#         skiplst.append(img)
#         continue

#     dfpth, df = tsgen.update_csvs(csv_path=img,
#                                   savepath=out_dir,
#                                   threshold=0.2)
    
    if len(df) < 1:
        print(f'Skipping empty CSV: {dfpth}')
        skiplst.append(img)
        continue
    else:
        donelst.append(img)
    
    
    # fig params -------------------------------------------------
    figdate = os.path.basename(img).split('____')[1].split('_')[0]
    figtitl = os.path.basename(out_dir)+'_'+figdate
    savpt_sctr = os.path.join(img_dir,figdate+'_0.png')
    savpt_rrs = os.path.join(img_dir,figdate+'_1.png')
    
    print(f'Generating image: {savpt_sctr}')
    
    # generate plot v1 --------------------------
#     tsgen.plot_single_sktr(
#         xdata=df['Oa08_reflectance:float'],
#         ydata=df['Oa21_reflectance:float'],
#         xlabel='Oa08: 665 nm',
#         ylabel='Oa17: 865 nm',
#         color=df['T865:float'],
#         clabel='T865',
#         title=figtitl,
#         savepathname=savpt)
    
    # generate plot v3 --------------------------
    tsgen.plot_sidebyside_sktr(x1_data=df['Oa08_reflectance:float'],
                               y1_data=df['Oa17_reflectance:float'],
                               x2_data=df['Oa08_reflectance:float'],
                               y2_data=df['Oa17_reflectance:float'],
                               x_lbl='RED: Oa08 (665nm)',
                               y_lbl='NIR: Oa17 (865nm)',
                               c1_data=df['A865:float'],
                               c1_lbl='Aer. Angstrom Expoent (A865)',
                               c2_data=df['T865:float'],
                               c2_lbl='Aer. Optical Thickness (T865)',
                               title=f'{os.path.basename(out_dir)} WFR {figdate} RED:Oa08(665nm) x NIR:Oa17(865nm)',
                               savepathname=savpt_sctr)
    
    print(f'Generating image: {savpt_rrs}')
    
    tsgen.s3l2_custom_reflectance_plot(df=df,
                                       figure_title=figdate,
                                       c_lbl='Aer. Optical Thickness (T865)',
                                       save_title=savpt_rrs)
    
t2 = time.perf_counter()
outputstr = f'>>> Finished in {round(t2 - t1, 2)} second(s). <<<'
print(outputstr)
os.system(f'telegram-send \"{outputstr}\"')

## Generating mean values dataset

In [ ]:
wdir = out_dir

todo = tsgen.build_list_from_subset(wdir)

todo_fullpath = [os.path.join(wdir,txt) for txt in todo]

## Converting and saving the list of mean values into a XLS excel file.

In [ ]:
data = tsgen.generate_time_series_datav2(wdir,todo)

series_df = pd.DataFrame(data=data)

path = '/d_drive_data/processing/linux/manacapuru_v10.xlsx'
# path = f'/d_drive_data/processing/linux/{os.path.basename(out_dir)}.xlsx'
# sheet = f'{os.path.basename(out_dir)}higlint'
sheet = f'v10g20'

book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book

# LINUX
series_df.to_excel(writer, sheet_name=sheet)

# WIN
# series_df.to_excel(f'D:\\processing\\win\\{os.path.basename(out_dir)}.xlsx', sheet_name=f'{os.path.basename(out_dir)}')

writer.save()
writer.close()

## CAMS+DBSCAN processing for a list of stations

In [ ]:
import netCDF4
from datetime import datetime
import geopandas as gpd

ncxp = NcExplorer(initialize=False, external_use=True)

def nc_dt2num(date):
    # https://stackoverflow.com/questions/39997314/write-times-in-netcdf-file
    units = "hours since 1900-01-01 00:00:00.0"
    calendar = "gregorian"
    return netCDF4.date2num(date, units=units, calendar=calendar)

def get_cams_band_by_time(numeric_date, cams_time_array):
    pos = np.where(cams_time_array == numeric_date)[0]
    return pos[0]

def find_cams_val(number_date, cams_time_array, query_lon=-60.8911,query_lat=-3.5726):
    # Manacapuru Centroid
    # query_lon, query_lat = -60.8911, -3.5726
    # query for the corret day position inside the CAMS time array
    day_position = get_cams_band_by_time(number_date, cams_time_array)
    # extract CAMS AOD data at this day
    aod_band = cams_nc.variables['aod865'][day_position][:]
    # query the AOD pixel value over the target coordinates
    cams_val = ncxp.get_point_data_in_single_band(aod_band, 
                                                 lon=lon_grid, 
                                                 lat=lat_grid,
                                                 target_lon=query_lon,
                                                 target_lat=query_lat)[1]
    
    return cams_val

In [ ]:
# Generating iterable list of NC paths

ncp = '/d_drive_data/CAMS/'
ncps = os.listdir(ncp)
ncps = [os.path.join(ncp,nc) for nc in ncps]

# Storing NC data inside dicts accessible by YEAR

lons = {}
lats = {}
cm_time = {}
cams_nc = {}
lon_grid = {}
lat_grid = {}

for ncp in ncps:
    
    print(f'extracting data for file: {ncp}')
    year = int(os.path.basename(ncp).split('.')[0].split('_')[1])
    cams_nc[year] = netCDF4.Dataset(ncp,'r')

    # get the longitude information
    lons[year] = cams_nc[year].variables['longitude'][:]
    # get the latitude information
    lats[year] = cams_nc[year].variables['latitude'][:]
    # extract the time dimension
    cm_time[year] = cams_nc[year].variables['time']

    # creating 2D grids for further plotting
    lon_grid[year], lat_grid[year] = np.meshgrid(lons[year], lats[year])

cm_time.keys()